In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch.optim import lr_scheduler
import torch.optim as optim
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import h5py as hp
from trainer import fit
import numpy as np
from torch.utils import data 
cuda = torch.cuda.is_available()

ECG_class=['N','V','L','R']

def load_mat(path_data,name_data,dtype='float32'):
    data=hp.File(path_data)
    arrays_d={}
    for k,v in data.items():
        arrays_d[k]=np.array(v)
    dataArr=np.array(arrays_d[name_data],dtype=dtype)
    return dataArr


ModuleNotFoundError: No module named 'torch'

## Prepare data and divide training and testing set

In [ ]:
Path='./' #自定义路径要正确
DataFile='Data_CNN.mat'
LabelFile='Label_OneHot.mat'
#print("Loading data and labels...")
Data=load_mat(Path+DataFile,'Data')
Label=load_mat(Path+LabelFile,'Label')
Data=Data.T
Indices=np.arange(Data.shape[0]) #随机打乱索引并切分训练集与测试集
np.random.shuffle(Indices)

#print("Divide training and testing set...")
train_x=Data[Indices[:5000]]
train_y=Label[Indices[:5000]]
test_x=Data[Indices[15000:]]
test_y=Label[Indices[15000:]]

train_y[0]


In [ ]:
hot_vectors=[]
for i in range(0,len(train_x)):
    list(train_x[i])
    hot_vectors.append(train_x[i])

labels=[];
for i in range(0,len(train_y)):
    list(train_y[i])
    labels.append(train_y[i])
#labels


In [ ]:
#from visualdl import LogWriter
#with LogWriter(logdir="./log/high_dimensional_test/train") as writer:
        # 将一组labels和对应的hot_vectors传入记录器进行记录
#        writer.add_embeddings(tag='default',
#                              labels=labels,
#                              hot_vectors=hot_vectors)

In [ ]:
class my_data(data.Dataset):
    def __init__(self, x, y,train, transform=None):
        self.xsignals =torch.from_numpy(x)
        self.ylabels = torch.from_numpy(y).max(dim=1)[1]
        self.train=train
        if transform is not None:   
            self.transform = transform
        else:
            self.transform = None

    def __getitem__(self, index):   #必须加载的方法
        signal = self.xsignals[index]
        Label = self.ylabels[index]
        return signal, Label   #返回处理完的图片数据和标签

    def __len__(self):     #必须加载的方法,实际上好像没什么用
        return len(self.ylabels)

In [ ]:
train_dataloader=torch.utils.data.DataLoader(dataset=my_data(train_x,train_y,1))
test_dataloader=torch.utils.data.DataLoader(dataset=my_data(test_x,test_y,0))
train_dataset=my_data(train_x,train_y,1)
test_dataset=my_data(test_x,test_y,0)
train_dataset.ylabels.shape
train_dataset.ylabels.shape
print(train_dataset.ylabels[0])
#torch.argmax(train_dataset.ylabels[0],-1)

In [ ]:
from torch.utils.data import Dataset
class SiameseECG(Dataset):
    """
    Train: For each sample creates randomly a positive or a negative pair
    Test: Creates fixed pairs for testing
    """

    def __init__(self, ecg_dataset):
        self.ecg_dataset = ecg_dataset

        self.train = self.ecg_dataset.train
        self.transform = self.ecg_dataset.transform

        if self.train:
            self.train_labels = self.ecg_dataset.ylabels
            self.train_data = self.ecg_dataset.xsignals
            self.labels_set = set(self.train_labels)
            self.label_to_indices = {label: np.where(self.train_labels == label)[0]
                                     for label in self.labels_set}
        else:
            # generate fixed pairs for testing
            self.test_labels = self.ecg_dataset.ylabels
            self.test_data = self.ecg_dataset.xsignals
            self.labels_set = list(set(self.test_labels))
            self.label_to_indices = {label: np.where(self.test_labels.numpy() == label)[0]
                                     for label in self.labels_set}
            print(self.test_labels)
            print(self.label_to_indices)
            print(self.label_to_indices[self.test_labels])
            random_state = np.random.RandomState(29)

            positive_pairs = [[i,
                               random_state.choice(self.label_to_indices[self.test_labels[i].item()]),
                               1]
                              for i in range(0, len(self.test_data), 2)]

            negative_pairs = [[i,
                               random_state.choice(self.label_to_indices[
                                                       np.random.choice(
                                                           list(self.labels_set - set([self.test_labels[i]]))
                                                       )
                                                   ]),
                               0]
                              for i in range(1, len(self.test_data), 2)]
            self.test_pairs = positive_pairs + negative_pairs

    def __getitem__(self, index):
        if self.train:
            target = np.random.randint(0, 2)
            img1, label1 = self.train_data[index], self.train_labels[index].item()
            if target == 1:
                siamese_index = index
                while siamese_index == index:
                    siamese_index = np.random.choice(self.label_to_indices[label1])
            else:
                siamese_label = np.random.choice(list(self.labels_set - set([label1])))
                siamese_index = np.random.choice(self.label_to_indices[siamese_label])
            img2 = self.train_data[siamese_index]
        else:
            img1 = self.test_data[self.test_pairs[index][0]]
            img2 = self.test_data[self.test_pairs[index][1]]
            target = self.test_pairs[index][2]


        if self.transform is not None:
            img1 = self.transform(img1)
            img2 = self.transform(img2)
        return (img1, img2), target

    def __len__(self):
        return len(self.ecg_dataset)

siamese_train_dataset = SiameseECG(train_dataset) # Returns pairs of images and target same/different
#siamese_test_dataset = SiameseECG(test_dataset)
batch_size = 128
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
siamese_train_loader = torch.utils.data.DataLoader(siamese_train_dataset, batch_size=batch_size, shuffle=True, **kwargs)
#siamese_test_loader = torch.utils.data.DataLoader(siamese_test_dataset, batch_size=batch_size, shuffle=False, **kwargs)

# Set up the network and training parameters
from networks import EmbeddingNet, SiameseNet
from loss import ContrastiveLoss

margin = 1.
embedding_net = EmbeddingNet()
model = SiameseNet(embedding_net)
if cuda:
    model.cuda()
loss_fn = ContrastiveLoss(margin)
lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)
n_epochs = 20
log_interval = 100

In [ ]:
from visualdl import LogWriter
with LogWriter(logdir="./log/scalar_test/train") as writer:
    for t in range(epochs):
        for step, (x, y) in enumerate(siamese_train_loader):
            # Forward pass: Compute predicted y by passing x to the model
            y_pred = model(x.cuda())

            # Compute and print loss
            loss = loss_fn(y_pred, y.cuda()) # 计算损失函数

            # Zero gradients, perform a backward pass, and update the weights.
            optimizer.zero_grad() # 梯度置零，因为反向传播过程中梯度会累加上一次循环的梯度
            loss.backward() # loss反向传播
            optimizer.step() # 反向传播后参数更新
            if(step%log_interval==0):
                print(loss)

            writer.add_scalar(tag="loss",step=step,value=loss)